In [1]:
import pandas as pd
import modules.graph as graph
import modules.geo as geo
import modules.db_management as dbm
import os

## Cálculo de estadísticas de solución antigua

In [30]:
data_path = os.path.join('Data', 'Colegios(Continental)2020.xlsx')
schools = pd.read_excel(data_path)

racs_path = os.path.join('Data', 'Raciones', 'Raciones_totales.xlsx')
racs = pd.read_excel(racs_path)

In [31]:
dupl_free = dbm.join_duplicate_schools(schools, coords_labels=['Longitud', 'Latitud'])
dupl_free['Logistica'] = pd.read_excel('logistics.xlsx')['Logistica']

Checking and joining duplicate schools: 100%|██████████| 8934/8934 [00:00<00:00, 28088.18it/s]


In [32]:
schools['RBD'] = schools['RBD'].apply(lambda x: [x])
dbm.add_food_rations_and_costs(schools, racs)
schools['RBD'] = schools['RBD'].apply(lambda x: x[0])

Calculating food rations and costs: 100%|██████████| 8934/8934 [00:00<00:00, 32197.48it/s]


In [33]:
new_logs = {'RBD': [], 'Logistica': []}
for i, row in dupl_free.iterrows():
    for rbd in row['RBD']:
        new_logs['RBD'].append(rbd)
        new_logs['Logistica'].append(row['Logistica'])

new_logs = pd.DataFrame(new_logs)

schools = schools.merge(new_logs, on='RBD', how='inner')

dbm.add_profit(schools)
ut_profits = dbm.get_ut_profits(schools, ut_label='UTE')
ut_profits = ut_profits[ut_profits['Raciones'] > 0].reset_index(drop=True)

ut_reassigned = {ut_profits['UTE'][i]: i for i in range(len(ut_profits))}

ut_profits['UTE'] = ut_profits['UTE'].apply(lambda x: ut_reassigned[x])
schools['UTE'] = schools['UTE'].apply(lambda x: ut_reassigned[x])


In [40]:
stats = dbm.obtain_stats(schools, ut_profits, ut_label='UTE', coords_labels=['Longitud', 'Latitud'])

En promedio, el beneficio económico entre UTs vecinas presenta una razón de 9.066
La razón más alta que se presenta entre UTs vecinas es 135.93
La menor razón que se presenta entre UTs vecinas es 1.006


## Cálculo de estadísticas de solución presentada

In [41]:
data_file = 'UT_profits.xlsx'
df_profits = pd.read_excel(data_file)

data_file = 'Colegios2020(UTs).xlsx'
df_schools = pd.read_excel(data_file)

In [42]:
stats = dbm.obtain_stats(df_schools, df_profits, ut_label='UT', coords_labels=['Longitud', 'Latitud'])

En promedio, el beneficio económico entre UTs vecinas presenta una razón de 1.287
La razón más alta que se presenta entre UTs vecinas es 2.325
La menor razón que se presenta entre UTs vecinas es 1.0
